In [12]:
#!/usr/bin/env python
#title           : Promoters1.0001
#description     : to design primers for promoters swap
#author          : Leo d'Espaux <leodespaux@gmail.com>,  
#date            : 20150423
#version         : 3.0
#notes           : includes options for 
#notes           : delGene replGene cutOligos
#notes           : at some point there will be one interface
#==============================================================================

# import libraries we're using

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.SeqUtils import MeltingTemp
import copy
from intermine.webservice import Service
from pandas import *
from pandas import DataFrame, read_csv
import pandas as pd #this is how I usually import pandas
import matplotlib.pyplot as plt
import sys


# define global variables
HomologyLength = 1000
PrimerMaxTm = 55 # As calculated by statluc, more like 65C by Thermo's Phusion program
PrimerMaxLen = 60
OverhangMaxFrac = 1

    
    
    
    
def PromOligos():

# This function asks user for a chromosomal locus, a region to be deleted, a suitable CRIPSR cutsite 
# and outputs oligos for cloning of a pL308 Cas9-gRNA vector, and ones for generating a donor DNA
# to delete the unwanted chromosomal region. Primers Lup+Rdown produce a 1kb band if deletion was
# successful. 
# part of yCRISPRv3 by leodespaux@gmail.com

    
    PromoterName=input("What's the name of the promoter? e.g. ADH1: ")
    PromoterGeneRec=fetchGene(PromoterName)
    PromoterRec=fetchNeighbor(PromoterGeneRec,"upstream",600)
    PromoterRec.name=PromoterRec.name+"ps"
    
    
    UpSeq="AGTAGCATCTGTGGCAACAGATTCTAGCCCTTTGAAGGATGAATGACTATTTTTTGTGCTAGTAGTGTAGAACACTTTATTTTGGGGAGCACTAGAGGAATATTGCGCCAACAATTGTGGGAGCGTCAAGAGGTCAGAAGCAGTCATTCGCCAATCTTTTGTTTTTCTCTGGTTCCCGTTCCCTACTGTTACTTTATTAGTATATAAGGGATCTGAATAAATATCAAGACGATGGAAAAAAAATAACAAGCAAAGGAACCTTTCTATATCCCAATTTTATTTCACATTAAAAACTATCGTCTTCCAACACTTTATATAACTGCTTGCAGCGGCCAGTAATTCGTCGCATCTCCATGACTCAAATTTTCCAGTGTCTCTTAGCAGTTAAACCATTCCTG"
    UpSeq=Seq(UpSeq)
    UpRec=SeqRecord(UpSeq, name="up")
                   
    DownSeq="ATGCAGATTTTCGTCAAGACTTTGACCGGTAAAACCATAACATTGGAAGTTGAATCTTCCGATACCATCGACAACGTTAAGTCGAAAATTCAAGACAAGGAAGGTATCCCTCCAGATCAACAAAGATTGATCTTTGCCGGTAAGCAGCTAGAAGACGGTAGAACGCTGTCTGATTACAACATTCAGAAGGAGTCCACCTTACATCTTGTGCTAAGGCTAAGAGGTGGTTATCACGGATCCGGAGCTTGGCTGTTGCCCGTCTCACTGGTGAAAAGAAAAACCACCCTGGCGCCCAATACGAGTAAAGGAGAAGAACTTTTCACTGGAGTTGTCCCAATTCTTGTTGAATTAGATGGTGATGTTAATGGGCACAAATTTTCT"
    DownSeq=Seq(DownSeq)
    DownRec=SeqRecord(DownSeq, name="down")
    
    fragments=[UpRec, PromoterRec, DownRec]
    
    stitch(fragments)
    
    
def stitch(fragments):
    #this function takes seq records and prints primers
    
    #let's make an empty sequence file
    Nfrags=len(fragments)
    donor=Seq("")
    index=[]
    print("")
    for i in range (0, Nfrags):
        donor=donor+fragments[i]
    
    for i in range (0, Nfrags):
        if i==0:
            print("Lup"+ fragments[i].name + " " + getPrimer(donor))
            print("Rup"+ fragments[i].name + "(" + fragments[i+1].name + ") " + overhangPrimer(fragments[i].reverse_complement(),fragments[i+1].reverse_complement()))
        elif i==Nfrags-1:
            print("Ldown"+ fragments[i].name + "(" + fragments[i-1].name + ") " + overhangPrimer(fragments[i],fragments[i-1]))
            print("Rdown"+ fragments[i].name + " " + getPrimer(donor.reverse_complement()))
        else:
            print("L"+ fragments[i].name + "(" + fragments[i-1].name + ") " + overhangPrimer(fragments[i],fragments[i-1]))
            print("R"+ fragments[i].name + "(" + fragments[i+1].name + ") " + overhangPrimer(fragments[i].reverse_complement(),fragments[i+1].reverse_complement()))

    print("")
    print("Your donor DNA cassette, has the following bp length and sequence:")


    print("")
    print(len(donor.seq))
    print("")

    print(donor.seq)

    print("")
    print("You might want to copy this entire prompt and save it for your records.")
    
        
def flipRecord(origRecord):
    origRecord.seq=origRecord.seq.reverse_complement()
    origRecord.name="r"+origRecord.name
    return origRecord
    
    
    
    
def fetchGene(GeneName):
    
    service = Service("http://yeastmine.yeastgenome.org/yeastmine/service")
    template = service.get_template('Gene_GenomicDNA')

    rows = template.rows(
        E = {"op": "LOOKUP", "value": GeneName, "extra_value": "S. cerevisiae"}
    )
    
    # this service seems to return multiple similar genes but we want the first one only, so count
    # and it returns information about the gene you want
    count=0
    for row in rows:
        
        count=count+1
        if count==1:
            descr= row["description"]
            GeneSeq=Seq(row["sequence.residues"])
            GeneSysName=row["secondaryIdentifier"]
       
    #let's create a record for the oldGene
    GeneRecord = SeqRecord(GeneSeq, id=GeneSysName)
    
    #now let's add some more information to make it useful
    GeneRecord.name=GeneName
    GeneRecord.features=GeneSysName
    GeneRecord.description=descr

    return GeneRecord 
    

def fetchNeighbor(NeighborRecord, direction, distance):

    # let's load the appropriate chromosome file. The record of the gene we looked up
    # contains in the "features" the systematic name, wherein the second letter
    # corresponds to chromosome number, e.g., 1=A etc
    if NeighborRecord.features[1]=="A":
        ChromosomeRec=SeqIO.read("Scer01.fasta", "fasta")
    if NeighborRecord.features[1]=="B":
        ChromosomeRec=SeqIO.read("Scer02.fasta", "fasta")
    if NeighborRecord.features[1]=="C":
        ChromosomeRec=SeqIO.read("Scer03.fasta", "fasta")
    if NeighborRecord.features[1]=="D":
        ChromosomeRec=SeqIO.read("Scer04.fasta", "fasta")
    if NeighborRecord.features[1]=="E":
        ChromosomeRec=SeqIO.read("Scer05.fasta", "fasta")
    if NeighborRecord.features[1]=="F":
        ChromosomeRec=SeqIO.read("Scer06.fasta", "fasta")
    if NeighborRecord.features[1]=="G":
        ChromosomeRec=SeqIO.read("Scer07.fasta", "fasta")
    if NeighborRecord.features[1]=="H":
        ChromosomeRec=SeqIO.read("Scer08.fasta", "fasta")
    if NeighborRecord.features[1]=="I":
        ChromosomeRec=SeqIO.read("Scer09.fasta", "fasta")
    if NeighborRecord.features[1]=="J":
        ChromosomeRec=SeqIO.read("Scer10.fasta", "fasta")
    if NeighborRecord.features[1]=="K":
        ChromosomeRec=SeqIO.read("Scer11.fasta", "fasta")
    if NeighborRecord.features[1]=="L":
        ChromosomeRec=SeqIO.read("Scer12.fasta", "fasta")
    if NeighborRecord.features[1]=="M":
        ChromosomeRec=SeqIO.read("Scer13.fasta", "fasta")
    if NeighborRecord.features[1]=="N":
        ChromosomeRec=SeqIO.read("Scer14.fasta", "fasta")
    if NeighborRecord.features[1]=="O":
        ChromosomeRec=SeqIO.read("Scer15.fasta", "fasta")
    if NeighborRecord.features[1]=="P":
        ChromosomeRec=SeqIO.read("Scer16.fasta", "fasta") 

    
    
    # let's explicitely name the sequences from the seq record
    NeighborSeq=NeighborRecord.seq
    ChromosomeSeq=ChromosomeRec.seq
    
    # flip the sequence to orient with respect to the old gene
    if ChromosomeSeq.find(NeighborSeq)==-1:
        ChromosomeSeq=ChromosomeSeq.reverse_complement()

    StartIndex=ChromosomeSeq.find(NeighborSeq)
    EndIndex=StartIndex+len(NeighborSeq)
    
    if direction=="upstream":
        DesiredSeq=ChromosomeSeq[StartIndex-distance:StartIndex]
    if direction=="downstream":
        DesiredSeq=ChromosomeSeq[EndIndex:EndIndex+distance]

    
    
    
    NeighborRec = SeqRecord(DesiredSeq, name=NeighborRecord.name)
    
    return NeighborRec

    
def getPrimer(currRecord):
    

    mp = 0
    length = 0
    primer = Seq("")

    seq=currRecord.seq
    
    while mp <= PrimerMaxTm and length <= PrimerMaxLen:
        primer = primer + seq[length]
        mp = MeltingTemp.Tm_staluc(primer)
        length += 1

    return primer           
        
def overhangPrimer(currRecord,prevSeq):
    #let's get the template-binding primer first
    primer=getPrimer(currRecord)
    
    
    #OK let's work on the overhang
    maxOhLen=PrimerMaxLen-len(primer)    
    maxFrac=1
    
    #let's decide on a max overhang length
    if round(len(primer)*(OverhangMaxFrac+1)) < 60:
             maxOhLen=round(len(primer)*OverhangMaxFrac)
    
    #the index must be an integer!!!
    maxOhLen=int(maxOhLen)
    ohprimer=prevSeq.seq[-maxOhLen:]+primer #we add the .seq so that it returns a string
    
    return ohprimer      
        
def standardCassette():
    
    #first, the promoter
    print("I'm going to build a standard cassette in which promoter is 600nt, terminator 250nt.")
    print("")
    print("Which PROMOTER do you want to use, e.g., TDH3")
    
    PromoterName=input("Your answer: ")
    PromoterGeneRec=fetchGene(PromoterName)
    PromoterRec=fetchNeighbor(PromoterGeneRec,"upstream",600)
    PromoterRec.name=PromoterRec.name+"ps"
    
    
    #second, the terminator
    print("Which TERMINATOR do you want to use, e.g., ADH1")
    TerminatorName = input('Your answer: ')
    TerminatorGeneRec=fetchGene(TerminatorName)
    TerminatorRec=fetchNeighbor(TerminatorGeneRec,"downstream",250)
    TerminatorRec.name=TerminatorRec.name+"ts"
    
    
    #and last, the gene
    print("What do you want to call the CDS you're inserting?")
    orfName = input("Your answer: ")
    
    print("What's the sequence")
    orfSeq=input("Your answer: ")
    
    orfRecord=SeqRecord(Seq(orfSeq), name=orfName)
    
    insertRec=[PromoterRec,orfRecord,TerminatorRec]
    return PromoterRec, orfRecord, TerminatorRec
          
def buildCassette():
    
    #first, the promoter
    print("I'm going to build a standard cassette in which promoter is 600nt, terminator 250nt.") 
    print("First, which PROMOTER do you want to use, e.g., TDH3")
    
    PromoterName=input("Your answer: ")
    PromoterGeneRec=fetchGene(PromoterName)
    PromoterRec=fetchNeighbor(PromoterGeneRec,"upstream",600)
    PromoterRec.name=PromoterRec.name+"ps"
    
    
    #second, the terminator
    print("Which TERMINATOR do you want to use, e.g., ADH1")
    TerminatorName = input('Your answer: ')
    TerminatorGeneRec=fetchGene(TerminatorName)
    TerminatorRec=fetchNeighbor(TerminatorGeneRec,"downstream",250)
    TerminatorRec.name=TerminatorRec.name+"ts"
    
    #and last, the gene
    print("What is the name of your gene, e.g., KlGapDH")
    orfName = input("Your answer: ")
    
    print("What's the sequence")
    orfSeq=input("Your answer: ")
    
    orfRecord=SeqRecord(Seq(orfSeq), name=orfName)
    
    insertRec=[PromoterRec,orfRecord,TerminatorRec]
    return PromoterRec, orfRecord, TerminatorRec
    

# now to run, type the program and ^Enter or play. E.g. remove hash below to run delGene()

PromOligos()



What's the name of the promoter? e.g. ADH1: TEF2

Lupup AGTAGCATCTGTGGCAACAGATTC
Rupup(TEF2ps) TATATATACATTAAACATTATTGGTAACAGGAATGGTTTAACTGCTAAGAGACA
LTEF2ps(up) TAAACCATTCCTGTTACCAATAATGTTTAATGTATATATATATATATATATATGGGGCCG
RTEF2ps(down) AAGTCTTGACGAAAATCTGCATGTTTAGTTAATTATAGTTCGTTGACCGTATATTCTAAA
Ldowndown(TEF2ps) CGGTCAACGAACTATAATTAACTAAACATGCAGATTTTCGTCAAGACTTTGACC
Rdowndown AGAAAATTTGTGCCCATTAACATCACC

Your donor DNA cassette, has the following bp length and sequence:

1379

AGTAGCATCTGTGGCAACAGATTCTAGCCCTTTGAAGGATGAATGACTATTTTTTGTGCTAGTAGTGTAGAACACTTTATTTTGGGGAGCACTAGAGGAATATTGCGCCAACAATTGTGGGAGCGTCAAGAGGTCAGAAGCAGTCATTCGCCAATCTTTTGTTTTTCTCTGGTTCCCGTTCCCTACTGTTACTTTATTAGTATATAAGGGATCTGAATAAATATCAAGACGATGGAAAAAAAATAACAAGCAAAGGAACCTTTCTATATCCCAATTTTATTTCACATTAAAAACTATCGTCTTCCAACACTTTATATAACTGCTTGCAGCGGCCAGTAATTCGTCGCATCTCCATGACTCAAATTTTCCAGTGTCTCTTAGCAGTTAAACCATTCCTGTTACCAATAATGTTTAATGTATATATATATATATATATATGGGGCCGTATACTTACATATAGTAGATGTCAAGCGTAGGCGCTTCCCCTGCCGGCTGTGAGGGCGCCATAACCAAGGT